# 載入套件、設定function、載入模型

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os, time, torchprofile
import torch.nn.functional as F
# from residual_attention_network import ResidualAttentionModel_92 as ResidualAttentionModel
from residual_attention_network import ResidualAttentionModel_2Layer as ResidualAttentionModel
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
import torch.optim as optim

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_classes = 50
epochs = 30
batch_size = 32
# test_batch_size = 32
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
# Define a custom dataset class
class ImageNetDataset(Dataset):
    def __init__(self, data_file, root_dir, transform=None):
        """
        Args:
            data_file (string): Path to the file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        # Read the data file
        with open(data_file, 'r') as file:
            self.data = file.readlines()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Process each line
        line = self.data[idx].strip()
        img_path, label = line.split()
        # Complete image path
        img_path = os.path.join(self.root_dir, img_path)
        image = Image.open(img_path).convert('RGB')
        label = int(label)
        
        if self.transform:
            image = self.transform(image)

        return image, label
    
# Define transformations

#基礎影像處理
basic_augmentations = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
train_augmentations = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(1),
    transforms.RandomVerticalFlip(0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
val_augmentations = transforms.Compose([
    transforms.Resize(384),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
test_augmentations = transforms.Compose([
    transforms.Resize(384),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])    

Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\wen\anaconda3\envs\wen\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\wen\anaconda3\envs\wen\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\wen\anaconda3\envs\wen\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\wen\anaconda3\envs\wen\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\wen\anaconda3\envs\wen\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 6: invalid start byte


# Loading Data

In [2]:
# Create dataset instances
train_dataset = ImageNetDataset(data_file='train.txt', root_dir='', transform=train_augmentations)
val_dataset = ImageNetDataset(data_file='val.txt', root_dir='', transform=val_augmentations)
test_dataset = ImageNetDataset(data_file='test.txt', root_dir='', transform=test_augmentations)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Loading Model

In [3]:
model = ResidualAttentionModel()
model.to(device)
print(model)

ResidualAttentionModel_2Layer(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (mpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (residual_block1): ResidualBlock(
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv4): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

# 訓練模型

In [4]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Change lr:'+str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, target) in train_loader_iter:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss=loss.item(), accuracy=100. * train_acc.item() / train_loader_len)
    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))


def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total=len(val_loader), desc="Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss=test_loss / len(val_loader.dataset), accuracy=100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))

    return correct / len(val_loader.dataset)


best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model,'best2_2layer.pt')
        print('Best Accuracy: {:.2f}%'.format(100. * best_val_acc))

print('Final Best Accuracy: {:.2f}%'.format(100. * best_val_acc))

Training Epoch #1: 100%|██████████| 1979/1979 [08:57<00:00,  3.68it/s, accuracy=11.1, loss=3.45]  


Train Epoch: 1, Loss: 3.436891, Accuracy: 11.12%


Validation Epoch #1: 100%|██████████| 15/15 [00:02<00:00,  6.86it/s, accuracy=15.8, loss=3.08] 


Validation Set: Average Loss: 3.0813, Accuracy: 15.78%
Best Accuracy: 15.78%


Training Epoch #2: 100%|██████████| 1979/1979 [08:14<00:00,  4.00it/s, accuracy=19, loss=3.11]  


...

Validation Set: Average Loss: 1.0065, Accuracy: 68.22%


Training Epoch #30: 100%|██████████| 1979/1979 [08:19<00:00,  3.96it/s, accuracy=63.3, loss=1.5]  


Train Epoch: 30, Loss: 1.239277, Accuracy: 63.31%


Validation Epoch #30: 100%|██████████| 15/15 [00:02<00:00,  7.39it/s, accuracy=70.9, loss=1]    


Validation Set: Average Loss: 1.0002, Accuracy: 70.89%
Best Accuracy: 70.89%
Final Best Accuracy: 70.89%


# 載入原始ResNet34

In [11]:
import torchvision.models as models

# 選擇ResNet-18作為模型
model = models.resnet34(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)  # CIFAR-10有10個類別
model.to(device)

c:\Users\wen\anaconda3\envs\wen\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# 訓練ResNet34

In [12]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Change lr:'+str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, target) in train_loader_iter:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss=loss.item(), accuracy=100. * train_acc.item() / train_loader_len)
    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))


def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total=len(val_loader), desc="Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss=test_loss / len(val_loader.dataset), accuracy=100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))

    return correct / len(val_loader.dataset)


best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model,'resnet34_best.pt')
        print('Best Accuracy: {:.2f}%'.format(100. * best_val_acc))

print('Final Best Accuracy: {:.2f}%'.format(100. * best_val_acc))

Training Epoch #1: 100%|██████████| 1979/1979 [04:48<00:00,  6.87it/s, accuracy=3.91, loss=3.6] 


Train Epoch: 1, Loss: 3.856757, Accuracy: 3.91%


Validation Epoch #1: 100%|██████████| 15/15 [00:01<00:00, 10.44it/s, accuracy=5.56, loss=3.69] 


Validation Set: Average Loss: 3.6922, Accuracy: 5.56%
Best Accuracy: 5.56%


Training Epoch #2: 100%|██████████| 1979/1979 [04:50<00:00,  6.82it/s, accuracy=7.12, loss=3.45]


...

Validation Set: Average Loss: 0.9913, Accuracy: 68.67%


Training Epoch #30: 100%|██████████| 1979/1979 [04:40<00:00,  7.05it/s, accuracy=66, loss=0.706]  


Train Epoch: 30, Loss: 1.134219, Accuracy: 66.04%


Validation Epoch #30: 100%|██████████| 15/15 [00:01<00:00, 10.86it/s, accuracy=68.2, loss=0.989]

Validation Set: Average Loss: 0.9894, Accuracy: 68.22%
Final Best Accuracy: 69.78%


In [2]:
ran_model = torch.load("best2_2layer.pt")
ran_model.to(device)
ran_model.eval()

resnet_model = torch.load("resnet34_best.pt")
resnet_model.to(device)
resnet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
test_augmentations = transforms.Compose([
    transforms.Resize(384),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_dataset = ImageNetDataset(data_file='test.txt', root_dir='', transform=test_augmentations)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 測試ResNet34

In [11]:
def test(model):
    model.eval()
    correct = 0
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total=len(test_loader), desc="Testing")
    
    all_preds = []
    all_targets = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for data, target in test_loader_iter:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            
            test_loader_iter.set_postfix(accuracy=100. * correct.item() / test_loader_len)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    accuracy = 100. * correct / test_loader_len
    
    # 計算 Precision, Recall 和 F1-score
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    
    # 計算 FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).to(device))
    
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

ran_acc, ran_precision, ran_recall, ran_f1, ran_flops, ran_elapsed_time = test(ran_model)
print(f"Accuracy: {ran_acc:.2f}%, Precision: {ran_precision:.4f}, Recall: {ran_recall:.4f}, F1 Score: {ran_f1:.4f}, FLOPS: {ran_flops:d}, Elapsed Time: {ran_elapsed_time:.2f} seconds")
resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(resnet_model)
print(f"Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.4f}, Recall: {resnet_recall:.4f}, F1 Score: {resnet_f1:.4f}, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:02<00:00,  3.69it/s, accuracy=66.4]


Accuracy: 66.44%, Precision: 0.6934, Recall: 0.6644, F1 Score: 0.6658, FLOPS: 3715245312, Elapsed Time: 2.17 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  5.90it/s, accuracy=69.6]

Accuracy: 69.56%, Precision: 0.7134, Recall: 0.6956, F1 Score: 0.6847, FLOPS: 3667013632, Elapsed Time: 1.36 seconds
